In [2]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset,random_split
import torchvision as tv
import numpy as np
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

import glob as gb

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.jso

chmod: cannot access '/root/.kaggle/kaggle.jso': No such file or directory


In [4]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
  0% 0.00/66.0M [00:00<?, ?B/s]
100% 66.0M/66.0M [00:00<00:00, 1.74GB/s]


In [5]:
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [6]:
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
df=pd.read_csv("/content/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [11]:
x

array([[ 0.00000000e+00, -1.35980713e+00, -7.27811733e-02, ...,
         1.33558377e-01, -2.10530535e-02,  1.49620000e+02],
       [ 0.00000000e+00,  1.19185711e+00,  2.66150712e-01, ...,
        -8.98309914e-03,  1.47241692e-02,  2.69000000e+00],
       [ 1.00000000e+00, -1.35835406e+00, -1.34016307e+00, ...,
        -5.53527940e-02, -5.97518406e-02,  3.78660000e+02],
       ...,
       [ 1.72788000e+05,  1.91956501e+00, -3.01253846e-01, ...,
         4.45477214e-03, -2.65608286e-02,  6.78800000e+01],
       [ 1.72788000e+05, -2.40440050e-01,  5.30482513e-01, ...,
         1.08820735e-01,  1.04532821e-01,  1.00000000e+01],
       [ 1.72792000e+05, -5.33412522e-01, -1.89733337e-01, ...,
        -2.41530880e-03,  1.36489143e-02,  2.17000000e+02]])

In [12]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)
x_train

array([[ 1.43352000e+05,  1.95504092e+00, -3.80782711e-01, ...,
         4.51682478e-02, -4.71447917e-02,  9.99000000e+00],
       [ 1.17173000e+05, -4.00975239e-01, -6.26942769e-01, ...,
        -3.70468822e-01, -1.44791686e-01,  4.59000000e+01],
       [ 1.49565000e+05,  7.25090164e-02,  8.20565650e-01, ...,
         2.06394866e-01,  7.02877702e-02,  1.19900000e+01],
       ...,
       [ 7.97950000e+04, -1.46608925e-01,  9.92946123e-01, ...,
        -1.21139194e-01, -1.96195328e-01,  3.94000000e+00],
       [ 8.79310000e+04, -2.94863809e+00,  2.35484929e+00, ...,
         4.96912107e-01,  3.35821632e-01,  1.00000000e+00],
       [ 7.63810000e+04,  1.23317435e+00, -7.84850501e-01, ...,
         1.21657270e-03,  3.85878912e-02,  1.13000000e+02]])

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit_transform(x_train)
sc.transform(x_test)

array([[-1.122118  , -8.43953559,  5.18252006, ..., -5.02350367,
        -3.16294175,  1.0815138 ],
       [-1.06408273,  0.1730534 , -1.65602032, ...,  0.10210496,
         0.30863538,  1.69317258],
       [-1.24890694,  0.71421679, -0.35634288, ...,  0.02847206,
         0.013272  , -0.22547388],
       ...,
       [ 1.00442384,  0.02018331, -0.06601034, ...,  0.89063984,
         1.0738269 , -0.31714621],
       [-0.75040591, -0.25325795,  0.59873588, ...,  0.10094377,
        -0.81883767, -0.33299371],
       [-0.96715373, -0.81263064,  0.59930001, ...,  0.31303049,
        -1.8421089 ,  3.2080607 ]])

In [15]:
df["Class"].nunique()

2

In [17]:
x_train.shape

(227845, 30)

In [46]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):

    self.features=features
    self.labels=labels

  def __len__(self):
    return len(self.labels)
  def __getitem__(self,idx):
     x = torch.tensor(self.features[idx], dtype=torch.float32)
     y = torch.tensor(self.labels[idx], dtype=torch.long)
     return x,y

In [47]:
train_ds=CustomDataset(x_train,y_train)
test_ds=CustomDataset(x_test,y_test)


In [56]:
train=DataLoader(
    train_ds,
    batch_size=32,
    shuffle=True,
    pin_memory=True
)
test=DataLoader(
    test_ds,
    batch_size=32,
    shuffle=False,
    pin_memory=True
)

In [57]:
class MYNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),

        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128,64),

        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,2),



    )

  def forward(self,x):
    return self.model(x)

In [59]:
epochs=100
learning_rate=0.1
model=MYNN(x_train.shape[1])
model.to(device)
criterion=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [60]:
!pip install torchinfo
from torchinfo import summary

In [61]:
summary(model,input_size=(32,x_train.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
MYNN                                     [32, 2]                   --
├─Sequential: 1-1                        [32, 2]                   --
│    └─Linear: 2-1                       [32, 128]                 3,968
│    └─BatchNorm1d: 2-2                  [32, 128]                 256
│    └─ReLU: 2-3                         [32, 128]                 --
│    └─Dropout: 2-4                      [32, 128]                 --
│    └─Linear: 2-5                       [32, 64]                  8,256
│    └─BatchNorm1d: 2-6                  [32, 64]                  128
│    └─ReLU: 2-7                         [32, 64]                  --
│    └─Dropout: 2-8                      [32, 64]                  --
│    └─Linear: 2-9                       [32, 2]                   130
Total params: 12,738
Trainable params: 12,738
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.41
Input size (MB): 0.00
Forward/backwa

In [62]:
from tqdm import tqdm

In [65]:
for epoch in range(epochs):

 model.train()
 total_loss=0
 correct=0
 total=0
 for x,y in tqdm (train,desc=f"epoch={epoch+1}"):
    x,y=x.to(device),y.to(device)
    output=model(x)
    loss=criterion(output,y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    preds = output.argmax(dim=1)
    correct += (preds == y).sum().item()
    total += y.size(0)
    total_loss+=loss.item()
 print(f"epoch;{epoch+1},Loss:{total_loss/len(train)}")


epoch=1: 100%|██████████| 7121/7121 [00:19<00:00, 363.19it/s]


epoch;1,Loss:0.013150730486391201


epoch=2: 100%|██████████| 7121/7121 [00:20<00:00, 354.58it/s]


epoch;2,Loss:0.01356286367788167


epoch=3: 100%|██████████| 7121/7121 [00:19<00:00, 356.79it/s]


...

epoch;98,Loss:0.013132503731050703


epoch=99: 100%|██████████| 7121/7121 [00:19<00:00, 371.51it/s]


epoch;99,Loss:0.013105311334546708


epoch=100: 100%|██████████| 7121/7121 [00:19<00:00, 361.45it/s]

epoch;100,Loss:0.01303852315207372


In [66]:
model.eval()

MYNN(
  (model): Sequential(
    (0): Linear(in_features=30, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [67]:

total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    output= model(batch_features)
    _, predicted=torch.max(output,1)
    total=total+batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
print(correct)
print("Accuracy:", correct / total)

56864
Accuracy: 0.9982795547909132
